<a href="https://colab.research.google.com/github/chi-yan/notebooks/blob/master/Classifier_builder_with_PyCaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Multi-label classification with pycaret

Code predicts "JOB CATEGORY" from a given description

Pycaret used to evaluate different models

Data source: https://data.world/promptcloud/30000-job-postings-from-seek-australia (hope's that ok...)

To use:
- download seek_australia.csv from https://data.world/promptcloud/30000-job-postings-from-seek-australia and save to Google Drive 



In [3]:
#https://data.world/promptcloud/30000-job-postings-from-seek-australia

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/seek_australia.csv')


In [ ]:
#drop rows where either category or description is NAN

In [5]:
data.dropna(subset = ["category","job_description"], inplace=True)
data['category'] = data['category'].astype('category')


In [6]:
categories = list(data['category'].cat.codes)

In [7]:
dict( enumerate(data['category'].cat.categories ) )
d = dict( enumerate(data['category'].cat.categories ) )
print(d)

{0: 'Accounting', 1: 'Administration & Office Support', 2: 'Advertising, Arts & Media', 3: 'Banking & Financial Services', 4: 'CEO & General Management', 5: 'Call Centre & Customer Service', 6: 'Community Services & Development', 7: 'Construction', 8: 'Consulting & Strategy', 9: 'Design & Architecture', 10: 'Education & Training', 11: 'Engineering', 12: 'Farming, Animals & Conservation', 13: 'Government & Defence', 14: 'Healthcare & Medical', 15: 'Hospitality & Tourism', 16: 'Human Resources & Recruitment', 17: 'Information & Communication Technology', 18: 'Insurance & Superannuation', 19: 'Legal', 20: 'Manufacturing, Transport & Logistics', 21: 'Marketing & Communications', 22: 'Mining, Resources & Energy', 23: 'Real Estate & Property', 24: 'Retail & Consumer Products', 25: 'Sales', 26: 'Science & Technology', 27: 'Self Employment', 28: 'Sport & Recreation', 29: 'Trades & Services'}


In [8]:
descriptions = data['job_description']

In [9]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


In [10]:
len(categories)

29655

In [25]:
embeddings = embed(descriptions[0:2000]) #train set [0:1000], test [1000:2000]


In [39]:
x = embeddings
y = categories[0:2000]

In [40]:
import numpy as np

x_np = np.array(x)
y_np = np.array(y)

In [41]:
x_np

array([[-0.02446888, -0.06791085,  0.04574434, ...,  0.01000986,
        -0.06279212, -0.04679998],
       [ 0.02802006, -0.05871655,  0.0501109 , ..., -0.03657731,
        -0.05850007, -0.02305215],
       [-0.05585852, -0.06168897, -0.05547146, ...,  0.01977858,
        -0.06100541, -0.04146077],
       ...,
       [-0.04673658, -0.06431934, -0.01019536, ..., -0.00942795,
        -0.05208879, -0.05426764],
       [-0.04892089, -0.06434394, -0.01475394, ..., -0.01165515,
        -0.04504905, -0.05910653],
       [-0.00513437, -0.0624182 , -0.00131203, ..., -0.02005271,
        -0.05955759, -0.03865134]], dtype=float32)

In [ ]:
#embeddings = embed(list(descriptions[0:50]))

In [ ]:
!pip install scikit-multilearn

In [ ]:
# Import BinaryRelevance from skmultilearn
from skmultilearn.problem_transform import BinaryRelevance

# Import SVC classifier from sklearn
from sklearn.svm import SVC

# Setup the classifier
classifier = BinaryRelevance(classifier=SVC(), require_dense=[False,True]) #support vector classifier

# Train
classifier.fit(x_np,y_np)

# Predict
#y_pred = classifier.predict(X_test)

BinaryRelevance(classifier=SVC(), require_dense=[False, True])

In [ ]:
y_pred = classifier.predict(x_np)

In [ ]:
y_np

array([24, 13, 29, ...,  6, 13, 29])

In [ ]:
y_numpy = np.array(y_np)

In [ ]:
yl = list(y_numpy)  #OBSERVED LABELS in list form

In [ ]:
y_pred_dense = list(y_pred.todense().ravel().flatten())

In [ ]:
yp = y_pred_dense 
print(yp[0].tolist())

[[24, 17, 29, 29, 29, 13, 25, 6, 14, 17, 29, 20, 20, 11, 7, 29, 22, 1, 21, 11, 1, 20, 3, 21, 17, 14, 29, 20, 21, 20, 14, 20, 13, 29, 24, 10, 29, 29, 29, 7, 17, 13, 0, 16, 10, 0, 14, 29, 29, 25, 11, 23, 10, 7, 29, 6, 20, 10, 6, 29, 1, 28, 13, 5, 25, 19, 17, 5, 23, 29, 17, 14, 29, 11, 29, 14, 13, 14, 14, 5, 5, 20, 0, 29, 6, 10, 1, 20, 13, 29, 29, 0, 29, 16, 21, 20, 17, 13, 14, 29, 13, 1, 17, 21, 29, 25, 15, 9, 15, 14, 16, 17, 14, 1, 10, 22, 14, 17, 14, 13, 24, 17, 1, 0, 15, 29, 11, 7, 21, 29, 17, 29, 1, 17, 29, 12, 3, 5, 0, 10, 20, 29, 29, 13, 3, 15, 6, 24, 1, 17, 25, 17, 20, 1, 0, 1, 23, 20, 17, 14, 9, 5, 1, 16, 11, 14, 13, 14, 21, 22, 17, 17, 13, 6, 29, 1, 29, 14, 15, 10, 21, 14, 7, 0, 3, 29, 13, 23, 14, 13, 3, 0, 15, 11, 13, 20, 22, 24, 29, 0, 13, 29, 18, 24, 21, 0, 14, 17, 15, 17, 25, 10, 25, 20, 1, 20, 14, 1, 29, 7, 29, 1, 14, 13, 14, 17, 1, 24, 14, 7, 13, 20, 1, 29, 7, 13, 17, 14, 10, 7, 29, 13, 13, 1, 29, 21, 25, 20, 23, 11, 25, 13, 25, 15, 16, 12, 17, 17, 13, 1, 17, 19, 15, 17, 2

In [ ]:
ypp = yp[0].reshape(-1,).tolist()[0] #PREDICTED LABELS in list form

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
matrix = classification_report(yl,ypp)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(matrix)

              precision    recall  f1-score   support

           0       0.89      0.92      0.90       542
           1       0.77      0.79      0.78       592
           2       1.00      0.41      0.58        49
           3       0.84      0.77      0.80       239
           4       1.00      0.12      0.21        25
           5       0.69      0.65      0.67       228
           6       0.87      0.85      0.86       247
           7       0.82      0.77      0.79       465
           8       1.00      0.17      0.29        41
           9       0.93      0.78      0.85       104
          10       0.88      0.94      0.91       422
          11       0.86      0.69      0.77       248
          12       1.00      0.82      0.90        40
          13       0.74      0.76      0.75       543
          14       0.85      0.94      0.89       783
          15       0.96      0.94      0.95       529
          16       0.86      0.77      0.81       237
          17       0.88    

In [42]:
x_df = pd.DataFrame(x_np)
y_df = pd.DataFrame(y_np,columns=["Target"])
data = pd.concat([x_df, y_df], axis = 1)

In [43]:
data

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,Target
0,-0.024469,-0.067911,0.045744,0.066710,0.010599,0.045103,-0.009321,-0.060522,0.024939,-0.067728,...,-0.067915,-0.064474,0.005024,-0.000441,-0.007278,0.019071,0.010010,-0.062792,-0.046800,24
1,0.028020,-0.058717,0.050111,0.052900,0.035309,-0.060094,0.043568,0.035527,-0.020640,-0.052010,...,-0.060984,-0.028657,-0.044012,0.040126,-0.017053,0.060536,-0.036577,-0.058500,-0.023052,13
2,-0.055859,-0.061689,-0.055471,0.056933,-0.021363,0.032672,0.049631,-0.056552,0.051257,-0.058588,...,-0.062195,-0.040162,0.017052,0.052379,-0.054074,0.054586,0.019779,-0.061005,-0.041461,29
3,-0.038752,-0.062129,0.044458,0.063895,0.023531,-0.061367,0.046407,-0.064515,0.037490,-0.066919,...,-0.067053,-0.035343,-0.025864,0.034938,-0.036394,0.025423,0.031487,-0.044204,0.009175,29
4,-0.003365,-0.098785,-0.013698,0.046806,-0.050783,0.074001,0.020125,-0.053147,0.085375,-0.082180,...,-0.053383,-0.039024,0.068907,0.062657,-0.040751,0.024947,-0.009998,-0.062230,0.006362,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-0.004934,-0.058724,-0.035293,0.053408,0.025101,-0.046209,-0.014525,-0.040520,-0.019289,-0.058615,...,-0.058752,0.034968,-0.046416,-0.028851,-0.054875,0.056423,-0.055810,-0.057836,-0.006508,29
1996,-0.057215,-0.056562,0.026559,-0.028688,0.020597,0.049384,-0.003532,-0.064197,0.064819,-0.066675,...,-0.066930,-0.044726,0.057451,0.060722,0.011680,0.042553,-0.025052,-0.058025,-0.001279,15
1997,-0.046737,-0.064319,-0.010195,0.062534,0.055854,-0.049461,0.054452,-0.041920,0.004667,-0.062512,...,-0.064335,0.037615,-0.006894,0.024548,-0.056302,-0.037179,-0.009428,-0.052089,-0.054268,20
1998,-0.048921,-0.064344,-0.014754,0.053352,0.032423,-0.010905,-0.003668,-0.050523,0.060635,-0.063233,...,-0.064410,-0.035750,0.018912,0.026009,-0.055371,-0.015097,-0.011655,-0.045049,-0.059107,29


In [4]:
#pip3 install jinja2
!pip3 install markupsafe==2.0.1
#!pip3 install --use-deprecated=legacy-resolver pycaret[full]


  Using cached MarkupSafe-2.0.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (31 kB)
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.1
    Uninstalling MarkupSafe-2.1.1:
      Successfully uninstalled MarkupSafe-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires markupsafe~=2.1.1, but you have markupsafe 2.0.1 which is incompatible.
pandas-profiling 3.2.0 requires PyYAML>=5.0.0, but you have pyyaml 3.13 which is incompatible.
pandas-profiling 3.2.0 requires requests>=2.24.0, but you have requests 2.23.0 which is incompatible.
flask-simplelogin 0.1.1 requires click<9.0.0,>=8.0.1, but you have click 7.1.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
autoviz 0.1.39 requires m

In [2]:

from pycaret.classification import *
import pycaret

In [1]:
3+3

6

     |████████████████████████████████| 53 kB 1.4 MB/s 
     |████████████████████████████████| 366 kB 9.7 MB/s 
     |████████████████████████████████| 305 kB 67.4 MB/s 
     |████████████████████████████████| 92 kB 116 kB/s 
     |████████████████████████████████| 132 kB 53.2 MB/s 
     |████████████████████████████████| 40 kB 5.3 MB/s 
     |████████████████████████████████| 76.6 MB 78.4 MB/s 
     |████████████████████████████████| 5.1 MB 27.0 MB/s 
     |████████████████████████████████| 308 kB 37.8 MB/s 
     |████████████████████████████████| 177 kB 43.9 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 100 kB 9.5 MB/s 
     |████████████████████████████████| 449 kB 47.3 MB/s 
     |████████████████████████████████| 11.9 MB 32.8 MB/s 
     |████████████████████████████████| 564 kB 50.7 MB/s 
     |████████████████████████████████| 53.2 MB 98 kB/s 
     |████████████████████████████████| 192.9 MB 52 kB/s 
     |█████████████████

In [32]:
clf = setup(data = data[0:1000], target = 'Target',session_id=1229,normalize=True,transformation=True,ignore_low_variance=True,
           remove_multicollinearity=True, multicollinearity_threshold=0.95)

,Description,Value
0,session_id,1229
1,Target,Target
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(1000, 513)"
5,Missing Values,False
6,Numeric Features,512
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='Target',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strat...
                ('dummy', Dummify(target='Target')),
                ('fix_perfect', Remove_100(target='Target')),
                ('clean_names', Clean_Colum_Names()),
     

In [33]:
compare_models(sort='Accuracy',budget_time=5, cross_validation=False)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.6578,0.0,0.5118,0.6811,0.6432,0.6313,0.6333,27.44
lr,Logistic Regression,0.6346,0.0,0.4980,0.6570,0.6309,0.6079,0.6093,4.60
xgboost,Extreme Gradient Boosting,0.6312,0.0,0.4921,0.6466,0.6159,0.6032,0.6060,12.39
svm,SVM - Linear Kernel,0.6080,0.0,0.4808,0.6282,0.6010,0.5790,0.5808,0.11
nb,Naive Bayes,0.6013,0.0,0.4630,0.6655,0.5984,0.5712,0.5743,0.01
knn,K Neighbors Classifier,0.5980,0.0,0.4770,0.6265,0.5845,0.5694,0.5716,0.03
rf,Random Forest Classifier,0.5781,0.0,0.4361,0.6338,0.5420,0.5409,0.5506,1.05
et,Extra Trees Classifier,0.5781,0.0,0.4172,0.6457,0.5297,0.5400,0.5522,0.75
gbc,Gradient Boosting Classifier,0.4850,0.0,0.3379,0.5178,0.4739,0.4448,0.4477,254.83
ridge,Ridge Classifier,0.4817,0.0,0.3751,0.5188,0.4822,0.4444,0.4455,0.03


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 2
INFO:logs:LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=1229, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
INFO:logs:compare_models() succesfully completed......................................


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=1229, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [34]:
lr = create_model('lr', fold =10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5857,0.0,0.5145,0.5671,0.5478,0.5540,0.5588
1,0.5714,0.0,0.5389,0.5956,0.5547,0.5425,0.5461
2,0.6429,0.0,0.5326,0.6079,0.6165,0.6159,0.6174
3,0.6286,0.0,0.5638,0.5779,0.5920,0.6007,0.6034
4,0.6286,0.0,0.5701,0.6084,0.5901,0.6013,0.6056
5,0.5857,0.0,0.5292,0.5580,0.5476,0.5555,0.5588
6,0.5286,0.0,0.4393,0.5698,0.5043,0.4943,0.4995
7,0.5571,0.0,0.4438,0.5112,0.5185,0.5263,0.5288
8,0.6143,0.0,0.5265,0.6322,0.5964,0.5873,0.5927


INFO:logs:create_model_container: 1
INFO:logs:master_model_container: 1
INFO:logs:display_container: 3
INFO:logs:LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1229, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
INFO:logs:create_model() succesfully completed......................................


In [35]:
tuned_lr= tune_model(lr) 

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5714,0.0,0.5097,0.5662,0.5412,0.5406,0.5439
1,0.5857,0.0,0.5806,0.6235,0.5767,0.5592,0.5630
2,0.6714,0.0,0.5951,0.6474,0.6505,0.6475,0.6489
3,0.6714,0.0,0.5958,0.6332,0.6356,0.6479,0.6514
4,0.6429,0.0,0.5785,0.6300,0.6110,0.6171,0.6209
5,0.6000,0.0,0.5431,0.5866,0.5685,0.5718,0.5753
6,0.5429,0.0,0.4593,0.5819,0.5219,0.5107,0.5143
7,0.5714,0.0,0.4630,0.5188,0.5324,0.5419,0.5441
8,0.6286,0.0,0.5342,0.6395,0.6081,0.6034,0.6078


INFO:logs:create_model_container: 2
INFO:logs:master_model_container: 2
INFO:logs:display_container: 4
INFO:logs:LogisticRegression(C=1.505, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1229, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
INFO:logs:tune_model() succesfully completed......................................


In [36]:
final_model = finalize_model(lr)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1229, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), fit_kwargs=None, groups=None, model_only=True, display=None, experiment_custom_tags=None, return_train_score=False)
INFO:logs:Finalizing LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1229, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
INFO:logs:Initializing create_model()
INFO:logs:create_model(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_i

In [45]:
unseen_predictions = predict_model(final_model, data=data[1000:2000]) 
unseen_predictions

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1229, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), probability_threshold=None, encoded_labels=False, drift_report=False, raw_score=False, round=4, verbose=True, ml_usecase=MLUsecase.CLASSIFICATION, display=None, drift_kwargs=None)
INFO:logs:Checking exceptions
INFO:logs:Preloading libraries
INFO:logs:Preparing display monitor


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.658,0,0.5143,0.6542,0.6495,0.6346,0.635


,0,1,2,3,4,5,6,7,8,9,...,505,506,507,508,509,510,511,Target,Label,Score
1000,-0.029878,-0.063449,0.024870,0.052027,0.012822,0.028136,0.007157,-0.056119,0.054748,-0.063421,...,0.004253,0.023157,-0.039500,0.030051,-0.002896,-0.061301,0.043527,24,24,0.9968
1001,-0.049897,-0.057055,0.022764,-0.012741,-0.003301,-0.057474,-0.032533,-0.050550,-0.056725,-0.057536,...,-0.056535,-0.006103,-0.047973,0.051386,0.031230,-0.057092,0.001202,13,13,0.9994
1002,0.024344,-0.064937,0.034295,-0.018015,0.035740,-0.054149,0.017965,-0.029214,-0.032238,-0.060777,...,-0.065524,-0.032749,-0.007258,0.048772,0.018746,-0.065368,0.037867,17,17,0.9892
1003,0.004884,-0.065356,0.027879,0.062380,0.003102,0.019415,0.056492,-0.064299,-0.060195,-0.064927,...,0.041333,0.046891,-0.041531,0.027016,-0.042452,-0.065611,-0.049095,5,25,0.9834
1004,-0.027524,-0.065379,-0.055730,0.062748,0.022067,-0.063120,0.049123,-0.049940,-0.023185,-0.066312,...,0.000391,0.056504,-0.063757,-0.004055,-0.055295,-0.057898,-0.036215,29,20,0.9367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-0.004934,-0.058724,-0.035293,0.053408,0.025101,-0.046209,-0.014525,-0.040520,-0.019289,-0.058615,...,-0.046416,-0.028851,-0.054875,0.056423,-0.055810,-0.057836,-0.006508,29,13,0.7858
1996,-0.057215,-0.056562,0.026559,-0.028688,0.020597,0.049384,-0.003532,-0.064197,0.064819,-0.066675,...,0.057451,0.060722,0.011680,0.042553,-0.025052,-0.058025,-0.001279,15,15,0.9869
1997,-0.046737,-0.064319,-0.010195,0.062534,0.055854,-0.049461,0.054452,-0.041920,0.004667,-0.062512,...,-0.006894,0.024548,-0.056302,-0.037179,-0.009428,-0.052089,-0.054268,20,20,0.9840
1998,-0.048921,-0.064344,-0.014754,0.053352,0.032423,-0.010905,-0.003668,-0.050523,0.060635,-0.063233,...,0.018912,0.026009,-0.055371,-0.015097,-0.011655,-0.045049,-0.059107,29,13,0.4429


In [46]:
data[1000:2000]

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,Target
1000,-0.029878,-0.063449,0.024870,0.052027,0.012822,0.028136,0.007157,-0.056119,0.054748,-0.063421,...,-0.063467,-0.040725,0.004253,0.023157,-0.039500,0.030051,-0.002896,-0.061301,0.043527,24
1001,-0.049897,-0.057055,0.022764,-0.012741,-0.003301,-0.057474,-0.032533,-0.050550,-0.056725,-0.057536,...,-0.057574,-0.035179,-0.056535,-0.006103,-0.047973,0.051386,0.031230,-0.057092,0.001202,13
1002,0.024344,-0.064937,0.034295,-0.018015,0.035740,-0.054149,0.017965,-0.029214,-0.032238,-0.060777,...,-0.066367,0.012648,-0.065524,-0.032749,-0.007258,0.048772,0.018746,-0.065368,0.037867,17
1003,0.004884,-0.065356,0.027879,0.062380,0.003102,0.019415,0.056492,-0.064299,-0.060195,-0.064927,...,-0.065992,-0.051910,0.041333,0.046891,-0.041531,0.027016,-0.042452,-0.065611,-0.049095,5
1004,-0.027524,-0.065379,-0.055730,0.062748,0.022067,-0.063120,0.049123,-0.049940,-0.023185,-0.066312,...,-0.066471,0.021064,0.000391,0.056504,-0.063757,-0.004055,-0.055295,-0.057898,-0.036215,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-0.004934,-0.058724,-0.035293,0.053408,0.025101,-0.046209,-0.014525,-0.040520,-0.019289,-0.058615,...,-0.058752,0.034968,-0.046416,-0.028851,-0.054875,0.056423,-0.055810,-0.057836,-0.006508,29
1996,-0.057215,-0.056562,0.026559,-0.028688,0.020597,0.049384,-0.003532,-0.064197,0.064819,-0.066675,...,-0.066930,-0.044726,0.057451,0.060722,0.011680,0.042553,-0.025052,-0.058025,-0.001279,15
1997,-0.046737,-0.064319,-0.010195,0.062534,0.055854,-0.049461,0.054452,-0.041920,0.004667,-0.062512,...,-0.064335,0.037615,-0.006894,0.024548,-0.056302,-0.037179,-0.009428,-0.052089,-0.054268,20
1998,-0.048921,-0.064344,-0.014754,0.053352,0.032423,-0.010905,-0.003668,-0.050523,0.060635,-0.063233,...,-0.064410,-0.035750,0.018912,0.026009,-0.055371,-0.015097,-0.011655,-0.045049,-0.059107,29
